In [1]:
import math 
from aocutils.common import to_int

lines = open('input.txt').read().splitlines()
lines = [to_int(line.split()) for line in lines]

def run(inp):
    reg = {'w': 0, 'x':0, 'y':0, 'z':0}
    inp_counter = 0
    for i in range(len(lines)):
        match lines[i]:
            case 'inp', register:
                reg[register] = inp[inp_counter]
                inp_counter += 1
            case 'add', rega, regb:
                if isinstance(regb, int):
                    reg[rega] += regb
                else:
                    reg[rega] += reg[regb]
            case 'mul', rega, regb:
                if isinstance(regb, int):
                    reg[rega] *= regb
                else:
                    reg[rega] *= reg[regb]
            case 'div', rega, regb:
                if isinstance(regb, int):
                    reg[rega] = math.floor(reg[rega] / regb)
                else:
                    reg[rega] = math.floor(reg[rega] / reg[regb])
            case 'mod', rega, regb:
                if isinstance(regb, int):
                    reg[rega] %= regb
                else:
                    reg[rega] %= reg[regb]
            case 'eql', rega, regb:
                if isinstance(regb, int):
                    if reg[rega] == regb:
                        reg[rega] = 1
                    else:
                        reg[rega] = 0
                else:
                    if reg[rega] == reg[regb]:
                        reg[rega] = 1
                    else:
                        reg[rega] = 0
            case _, _:
                print('not defined')
    return reg
inp = '74929995999389'
inp = [int(i) for i in inp]
%timeit run(inp)

60.8 µs ± 1.13 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [2]:
# these are the only 3 lines that are changing per block of input
div = ([lines[4+18*j][2] for j in range(14)])
add1 = ([lines[5+18*j][2] for j in range(14)])
add2 = ([lines[15+18*j][2] for j in range(14)])


def step(w,z,rnd):
    # this is what the program is doing simplified
    # input of round with div 26 needs to be (add1) + input
    # input of round with div 1 needs to be 0 and then the output is add2 + input
    x = (z % 26) + add1[rnd]
    z = z // div[rnd]
    if w != x:
        z *= 26
        z += w + add2[rnd]
    return z

def test():
    for i in range(13):
        step(1,0,i)
# it's a speedup of about 30x compared with the program, which we need
%timeit test()

2.88 µs ± 84.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [3]:
# first build for each round a list of valid inputs to produce a valid output
# this takes about 45 seconds, since we try almost 10 million times per round
roundoptions = [{0}]
for rnd in range(13,-1,-1):
    # we start at the end, since we know the output of the last round must be 0
    print('running round', rnd)
    options = roundoptions[-1]
    newoptions = set()
    for z in range(1000000):
        for w in range(1,10):
            res = step(w, z, rnd)
            if res in options:
                # this combination of w and z produces a valid output for the next round
                newoptions.add(z)                # 
                # print('input for ronde', ronde, 'needs to be', z, 'if input is', w, 'and res', res)
    roundoptions.append(newoptions)

running round 13
running round 12
running round 11
running round 10
running round 9
running round 8
running round 7
running round 6
running round 5
running round 4
running round 3
running round 2
running round 1
running round 0


In [4]:
# now we start from the beginning with z=0 as input
# try starting with the highest (p1) or lowest value (p2)
z = 0
options = list(reversed(roundoptions))[1:]
rnd = 0
ans = []
for r in options:
    for inp in range(9, 0, -1):
        res = step(inp, z, rnd)
        if res in r:
            print(f'Round {rnd}: z value {z}, input value {inp}')
            ans.append(inp)
            z = res
            break
    rnd += 1
''.join([str(i) for i in ans])     


Round 0: z value 0, input value 7
Round 1: z value 21, input value 4
Round 2: z value 558, input value 9
Round 3: z value 14521, input value 2
Round 4: z value 377558, input value 9
Round 5: z value 14521, input value 9
Round 6: z value 558, input value 9
Round 7: z value 14521, input value 5
Round 8: z value 558, input value 9
Round 9: z value 21, input value 9
Round 10: z value 0, input value 9
Round 11: z value 9, input value 3
Round 12: z value 0, input value 8
Round 13: z value 21, input value 9


'74929995999389'